# Advent of Code 2021
https://adventofcode.com/2021

# Utils

## Imports

In [ ]:
import sys

import numpy as np

## Parsing utilities

In [ ]:
def dayfile(daynum):
    with open(f"input{daynum}") as f:
        data = f.read().rstrip()
    return data

In [ ]:
def as_list(element_parser, **args):
    return lambda data: [element_parser(e) for e in data.split(**args) if e != ""]

In [ ]:
def as_matrix(element_parser, row_sep="\n", col_sep=""):
    def _inner(data):
        data = data.strip()
        res = []
        for e in data.split(row_sep):
            if col_sep == "":
                e = list(e)
            else:
                e = e.split(col_sep)
            res.append([element_parser(i.strip()) for i in e if i != ""])
        return np.asarray(res)
    return _inner

In [ ]:
def as_pair(element_parser, sep=","):
    def _inner(entry):
        a, b = entry.split(sep)
        return (element_parser(a), element_parser(b))
    return _inner

# Day 1: Sonar Sweep

As the submarine drops below the surface of the ocean, it automatically performs a sonar sweep of the nearby sea floor. On a small screen, the sonar sweep report (your puzzle input) appears: each line is a measurement of the sea floor depth as the sweep looks further and further away from the submarine.

For example, suppose you had the following report:

```
199
200
208
210
200
207
240
269
260
263
```

This report indicates that, scanning outward from the submarine, the sonar sweep found depths of 199, 200, 208, 210, and so on.

The first order of business is to figure out how quickly the depth increases, just so you know what you're dealing with - you never know if the keys will get carried into deeper water by an ocean current or a fish or something.

To do this, count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.) In the example above, the changes are as follows:

```
199 (N/A - no previous measurement)
200 (increased)
208 (increased)
210 (increased)
200 (decreased)
207 (increased)
240 (increased)
269 (increased)
260 (decreased)
263 (increased)
```

In this example, there are 7 measurements that are larger than the previous measurement.

How many measurements are larger than the previous measurement?

In [ ]:
parse = as_list(int)

In [ ]:
test_data = parse("""
199
200
208
210
200
207
240
269
260
263
""")

data = parse(dayfile(1))

In [ ]:
def d1_1(data):
    count = 0
    prev = data[0]
    for e in data[1:]:
        if e > prev:
            count += 1
        prev = e
    return count

In [ ]:
d1_1(test_data)

In [ ]:
d1_1(data)

## Part 2

Considering every single measurement isn't as useful as you expected: there's just too much noise in the data.

Instead, consider sums of a three-measurement sliding window. Again considering the above example:

```
199  A      
200  A B    
208  A B C  
210    B C D
200  E   C D
207  E F   D
240  E F G  
269    F G H
260      G H
263        H
```

Start by comparing the first and second three-measurement windows. The measurements in the first window are marked A (199, 200, 208); their sum is 199 + 200 + 208 = 607. The second window is marked B (200, 208, 210); its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.

Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare A with B, then compare B with C, then C with D, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.

In the above example, the sum of each three-measurement window is as follows:

```
A: 607 (N/A - no previous sum)
B: 618 (increased)
C: 618 (no change)
D: 617 (decreased)
E: 647 (increased)
F: 716 (increased)
G: 769 (increased)
H: 792 (increased)
```

In this example, there are 5 sums that are larger than the previous sum.

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [ ]:
def d1_2(data):
    count = 0
    prev = sum(data[0:3])
    for i in range(1, len(data) - 2):
        e = sum(data[i:(i + 3)])
        if e > prev:
            count += 1
        prev = e
    return count

In [ ]:
d1_2(test_data)

In [ ]:
d1_2(data)

## Generalizing

In [ ]:
def d1_n(data, sliding):
    count = 0
    prev = sum(data[0:sliding])
    for i in range(1, len(data) - sliding + 1):
        e = sum(data[i:(i + sliding)])
        if e > prev:
            count += 1
        prev = e
    return count

In [ ]:
assert d1_1(data) == d1_n(data, sliding=1)
assert d1_2(data) == d1_n(data, sliding=3)

# Day 2: Dive
Now, you need to figure out how to pilot this thing.

It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

- forward X increases the horizontal position by X units.
- down X increases the depth by X units.
- up X decreases the depth by X units.

Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.

The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

```
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

- forward 5 adds 5 to your horizontal position, a total of 5.
- down 5 adds 5 to your depth, resulting in a value of 5.
- forward 8 adds 8 to your horizontal position, a total of 13.
- up 3 decreases your depth by 3, resulting in a value of 2.
- down 8 adds 8 to your depth, resulting in a value of 10.
- forward 2 adds 2 to your horizontal position, a total of 15.

After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [ ]:
"test".split


In [ ]:
"test".split

In [ ]:
def move(entry):
    direction, amount = entry.split(" ")
    amount = int(amount)
    return direction, amount

parse = as_list(move, sep="\n")

In [ ]:
test_data = parse("""
forward 5
down 5
forward 8
up 3
down 8
forward 2
""")

data = parse(dayfile(2))

In [ ]:
def d2_1(moves):
    pos = 0
    depth = 0
    
    for direction, amount in moves:
       if direction == "forward":
            pos += amount
       if direction == "down":
            depth += amount
       if direction == "up":
            depth -= amount
    
    return pos * depth

In [ ]:
d2_1(test_data)

In [ ]:
d2_1(data)

## Part 2

Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:

- down X increases your aim by X units.
- up X decreases your aim by X units.
- forward X does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.

Now, the above example does something different:

- forward 5 adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
- down 5 adds 5 to your aim, resulting in a value of 5.
- forward 8 adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
- up 3 decreases your aim by 3, resulting in a value of 2.
- down 8 adds 8 to your aim, resulting in a value of 10.
- forward 2 adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.

After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [ ]:
def d2_2(moves):
    pos = 0
    depth = 0
    aim = 0
    
    for direction, amount in moves:
       if direction == "forward":
            pos += amount
            depth += aim * amount
       if direction == "down":
            aim += amount
       if direction == "up":
            aim -= amount
    
    return pos * depth

In [ ]:
d2_2(test_data)

In [ ]:
d2_2(data)

# Day 3: Binary Diagnostic

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.

So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [ ]:
parse = as_matrix(int)

In [ ]:
test_data = parse("""
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
""")

data = parse(dayfile(3))

In [ ]:
np.argmax(np.unique(test_data[:, 0], return_counts=True)[1])

In [ ]:
test_data.shape

In [ ]:
def d3_1(data):
    counts = [0 for _ in data[0]]
    gamma = []
    epsilon = []
    for i in range(data.shape[1]):
        most_common =  np.argmax(np.unique(data[:, i], return_counts=True)[1])
        least_common = 1 - most_common
        gamma.append(str(most_common))
        epsilon.append(str(least_common))
    gamma_rate = int(f"0b{''.join(gamma)}", 2)
    epsilon_rate = int(f"0b{''.join(epsilon)}", 2)
    
    return gamma_rate * epsilon_rate

In [ ]:
d3_1(test_data)

In [ ]:
d3_1(data)

## Part 2

Next, you should verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

- Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
- If you only have one number left, stop; this is the rating value for which you are searching.
- Otherwise, repeat the process, considering the next bit to the right.

The bit criteria depends on which type of rating value you want to find:

- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

For example, to determine the oxygen generator rating value using the same example diagnostic report from above:

- Start with all 12 numbers and consider only the first bit of each number. There are more 1 bits (7) than 0 bits (5), so keep only the 7 numbers with a 1 in the first position: 11110, 10110, 10111, 10101, 11100, 10000, and 11001.
- Then, consider the second bit of the 7 remaining numbers: there are more 0 bits (4) than 1 bits (3), so keep only the 4 numbers with a 0 in the second position: 10110, 10111, 10101, and 10000.
- In the third position, three of the four numbers have a 1, so keep those three: 10110, 10111, and 10101.
- In the fourth position, two of the three numbers have a 1, so keep those two: 10110 and 10111.
- In the fifth position, there are an equal number of 0 bits and 1 bits (one each). So, to find the oxygen generator rating, keep the number with a 1 in that position: 10111.
- As there is only one number left, stop; the oxygen generator rating is 10111, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

- Start again with all 12 numbers and consider only the first bit of each number. There are fewer 0 bits (5) than 1 bits (7), so keep only the 5 numbers with a 0 in the first position: 00100, 01111, 00111, 00010, and 01010.
- Then, consider the second bit of the 5 remaining numbers: there are fewer 1 bits (2) than 0 bits (3), so keep only the 2 numbers with a 1 in the second position: 01111 and 01010.
- In the third position, there are an equal number of 0 bits and 1 bits (one each). So, to find the CO2 scrubber rating, keep the number with a 0 in that position: 01010.
- As there is only one number left, stop; the CO2 scrubber rating is 01010, or 10 in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [ ]:
def d3_2(data):
    counts = [0 for _ in data[0]]
    oxygen_candidates = data
    for i in range(data.shape[1]):
        zeroes = oxygen_candidates[oxygen_candidates[:, i] == 0]
        ones = oxygen_candidates[oxygen_candidates[:, i] == 1]
        if ones.shape[0] >= zeroes.shape[0]:
            oxygen_candidates = ones
        else:
            oxygen_candidates = zeroes
        if oxygen_candidates.shape[0] == 1:
            break
    oxygen_rate = int(f"0b{''.join([str(e) for e in oxygen_candidates[0, ]])}", 2)
    
    co2_candidates = data
    for i in range(data.shape[1]):
        zeroes = co2_candidates[co2_candidates[:, i] == 0]
        ones = co2_candidates[co2_candidates[:, i] == 1]
        if zeroes.shape[0] <= ones.shape[0]:
            co2_candidates = zeroes
        else:
            co2_candidates = ones
        if co2_candidates.shape[0] == 1:
            break
    co2_rate = int(f"0b{''.join([str(e) for e in co2_candidates[0, ]])}", 2)
    return oxygen_rate * co2_rate

In [ ]:
d3_2(test_data)

In [ ]:
d3_2(data)

# Day 4: Giant Squid

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
 ```
 
 After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners.
After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners. Finally, 24 is drawn.

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

In [ ]:
def parse(data):
    s_data = data.split("\n\n")
    draw = as_list(int, sep=",")(s_data[0])
    boards = [as_matrix(int, col_sep=" ")(e) for e in s_data[1:]]
    return {
        "draw": draw,
        "boards": boards,
    }

In [ ]:
test_data = parse("""
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
""")

data = parse(dayfile(4))

In [ ]:
def is_win(board, draws):
    won = False
    for i in range(board.shape[0]):
        if np.in1d(board[i, :], draws).all():
            # Found a winning row
            return True
    for i in range(board.shape[1]):
        if np.in1d(board[:, i], draws).all():
            # Found a winning col
            return True
    return False

In [ ]:
def d4_1(data):
    for i in range(1, len(data["draw"])):
        curr_draw = data["draw"][0:i]
        for board in data["boards"]:
            if is_win(board, curr_draw):
                board_values = board.flatten()
                return np.sum(board_values[~np.isin(board_values, curr_draw)]) * curr_draw[-1]

In [ ]:
d4_1(test_data)

In [ ]:
d4_1(data)

## Part 2
On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?

In [ ]:
def d4_2(data):
    candidates = data["boards"].copy()
    for i in range(1, len(data["draw"])):
        curr_draw = data["draw"][0:i]
        for i, board in enumerate(candidates):
            if is_win(board, curr_draw):
                candidates.pop(i)
                if len(candidates) == 0:
                    board_values = board.flatten()
                    return np.sum(board_values[~np.isin(board_values, curr_draw)]) * curr_draw[-1]

In [ ]:
d4_2(test_data)

In [ ]:
d4_2(data)

# Day 5: Hydrothermal Venture

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

- An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
- An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```

In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [ ]:
def d4_line(data):
    start, stop = tuple(data.split(" -> "))
    return [
        as_list(int, sep=",")(start),
        as_list(int, sep=",")(stop),
    ]

parse = as_list(d4_line, sep="\n")

In [ ]:
test_data = parse("""
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
""")

data = parse(dayfile(5))

In [ ]:
def d5_1(data):
    sparse_grid = {}
    overlaps = set()
    for start, stop in data:
        x1, y1 = start
        x2, y2 = stop
        
        # check line is straight
        if x1 != x2 and y1 != y2:
            continue
        
        xb = min(x1, x2)
        xe = max(x1, x2) + 1
        yb = min(y1, y2)
        ye = max(y1, y2) + 1
        for i in range(xb, xe):
            for j in range(yb, ye):
                if (i, j) not in sparse_grid:
                    sparse_grid[(i, j)] = 1
                else:
                    sparse_grid[(i, j)] += 1
                    overlaps.add((i, j))
    return len(overlaps)

In [ ]:
d5_1(test_data)

In [ ]:
d5_1(data)

## Part 2

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

- An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
- An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:

```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [ ]:
def d5_2(data, out=sys.stdout):
    sparse_grid = {}
    overlaps = set()
    x_max = 0
    y_max = 0
    for start, stop in data:
        x1, y1 = start
        x2, y2 = stop
        
        x_max = max(x_max, max(x1, x2))
        y_max = max(y_max, max(y1, y2))
        
        if x1 == x2 or y1 == y2:
            xb = min(x1, x2)
            xe = max(x1, x2) + 1
            yb = min(y1, y2)
            ye = max(y1, y2) + 1
            values = [(x, y)
                      for x in range(xb, xe)
                      for y in range(yb, ye)]
        else:
            x_step = 1 if x1 <= x2 else -1
            y_step = 1 if y1 <= y2 else -1
            values = zip(
                range(x1, x2 + x_step, x_step),
                range(y1, y2 + y_step, y_step),
            )
        
        for (i, j) in values:
                if (i, j) not in sparse_grid:
                    sparse_grid[(i, j)] = 1
                else:
                    sparse_grid[(i, j)] += 1
                    overlaps.add((i, j))
    
    # Display the result
    for i in range(0, x_max + 1):
        for j in range(0, y_max + 1):
            if (j, i) not in sparse_grid:
                print(".", end="", file=out)
            else:
                print(sparse_grid[(j, i)], end="", file=out)
        
        print("", file=out)
    
    return len(overlaps)

In [ ]:
d5_2(test_data)

In [ ]:
with open("day5_2_out", "w") as out:
    print(d5_2(data, out))

# Day 6: Lanternfish

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

- After one day, its internal timer would become 2.
- After another day, its internal timer would become 1.
- After another day, its internal timer would become 0.
- After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
- After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.

A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

```
3,4,3,1,2
```

This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:

```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
```

Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

In [ ]:
parse = as_list(int, sep=",")

In [ ]:
test_data = parse("3,4,3,1,2")

data = parse(dayfile(6))

In [ ]:
def d6_1(data, days):
    data = data.copy()
    for day in range(days):
        for i, e in enumerate(data):
            if e == 0:
                data.append(9)
                e = 7
            data[i] = e - 1
    return len(data)

In [ ]:
d6_1(test_data, 18)

In [ ]:
d6_1(test_data, 80)

In [ ]:
d6_1(data, 80)

## Part 2

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of 26984457539 lanternfish!

How many lanternfish would there be after 256 days?

In [ ]:
def d6_2(data, days, debug = False):
    generations = {i:0 for i in range(7)}
    for i in data:
        generations[i + 1] += 1

    youngs_generations = {i:0 for i in range(7)}
    for day in range(days + 1):
        active_gen = day % 7
        mature_gen = (active_gen - 2) % 7
        
        new_youngs = generations[active_gen]
        new_matures = youngs_generations[mature_gen]
        youngs_generations[mature_gen] = 0

        youngs_generations[active_gen] += new_youngs
        generations[active_gen] += new_matures

        if debug:
            print("----------")
            print(f"Day {day} / {active_gen}")
            print(f"{generations=}")
            print(f"{youngs_generations=}")
            print(f"{active_gen=} / new youngs: {new_youngs} / new matures: {new_matures} / total matures: {sum(generations.values())} / total youngs: {sum(youngs_generations.values())}")
        
    return sum(generations.values()) + sum(youngs_generations.values())

In [ ]:
d6_2(test_data, 256)

In [ ]:
d6_2(data, 256)

# Day 7: The Treachery of Whales

A giant whale has decided your submarine is its next meal, and it's much faster than you are. There's nowhere to run!

Suddenly, a swarm of crabs (each in its own tiny submarine - it's too deep for them otherwise) zooms in to rescue you! They seem to be preparing to blast a hole in the ocean floor; sensors indicate a massive underground cave system just beyond where they're aiming!

The crab submarines all need to be aligned before they'll have enough power to blast a large enough hole for your submarine to get through. However, it doesn't look like they'll be aligned before the whale catches you! Maybe you can help?

There's one major catch - crab submarines can only move horizontally.

You quickly make a list of the horizontal position of each crab (your puzzle input). Crab submarines have limited fuel, so you need to find a way to make all of their horizontal positions match while requiring them to spend as little fuel as possible.

For example, consider the following horizontal positions:

```
16,1,2,0,4,2,7,1,2,14
```

This means there's a crab with horizontal position 16, a crab with horizontal position 1, and so on.

Each change of 1 step in horizontal position of a single crab costs 1 fuel. You could choose any horizontal position to align them all on, but the one that costs the least fuel is horizontal position 2:

- Move from 16 to 2: 14 fuel
- Move from 1 to 2: 1 fuel
- Move from 2 to 2: 0 fuel
- Move from 0 to 2: 2 fuel
- Move from 4 to 2: 2 fuel
- Move from 2 to 2: 0 fuel
- Move from 7 to 2: 5 fuel
- Move from 1 to 2: 1 fuel
- Move from 2 to 2: 0 fuel
- Move from 14 to 2: 12 fuel

This costs a total of 37 fuel. This is the cheapest possible outcome; more expensive outcomes include aligning at position 1 (41 fuel), position 3 (39 fuel), or position 10 (71 fuel).

Determine the horizontal position that the crabs can align to using the least fuel possible. How much fuel must they spend to align to that position?



In [ ]:
parse = as_list(int, sep=",")

In [ ]:
test_data = parse("16,1,2,0,4,2,7,1,2,14")

data = parse(dayfile(7))

In [ ]:
def d7_1(data):
    min_cost = None
    min_pos = None
    for i in range(min(data), max(data) + 1):
        total_cost = 0
        for e in data:
            total_cost += abs(e - i)
        if min_cost is None or min_cost > total_cost:
            min_cost = total_cost
            min_pos = i
    return min_pos, min_cost

In [ ]:
d7_1(test_data)

In [ ]:
d7_1(data)

## Part 2

The crabs don't seem interested in your proposed solution. Perhaps you misunderstand crab engineering?

As it turns out, crab submarine engines don't burn fuel at a constant rate. Instead, each change of 1 step in horizontal position costs 1 more unit of fuel than the last: the first step costs 1, the second step costs 2, the third step costs 3, and so on.

As each crab moves, moving further becomes more expensive. This changes the best horizontal position to align them all on; in the example above, this becomes 5:

- Move from 16 to 5: 66 fuel
- Move from 1 to 5: 10 fuel
- Move from 2 to 5: 6 fuel
- Move from 0 to 5: 15 fuel
- Move from 4 to 5: 1 fuel
- Move from 2 to 5: 6 fuel
- Move from 7 to 5: 3 fuel
- Move from 1 to 5: 10 fuel
- Move from 2 to 5: 6 fuel
- Move from 14 to 5: 45 fuel

This costs a total of 168 fuel. This is the new cheapest possible outcome; the old alignment position (2) now costs 206 fuel instead.

Determine the horizontal position that the crabs can align to using the least fuel possible so they can make you an escape route! How much fuel must they spend to align to that position?



In [ ]:
def d7_2(data):
    min_cost = None
    min_pos = None
    for i in range(min(data), max(data) + 1):
        total_cost = 0
        for e in data:
            dist = abs(e - i)
            total_cost += dist * (dist + 1) / 2
        if min_cost is None or min_cost > total_cost:
            min_cost = total_cost
            min_pos = i
    return min_pos, min_cost

In [ ]:
d7_2(test_data)

In [ ]:
d7_2(data)

# Day 8: Seven Segment Search

You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.

As your submarine slowly makes its way through the cave system, you notice that the four-digit seven-segment displays in your submarine are malfunctioning; they must have been damaged during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named a through g:

```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
```

So, to render a 1, only segments c and f would be turned on; the rest would be off. To render a 7, only segments a, c, and f would be turned on.

The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires a through g, but those wires are connected to segments randomly. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits within a display use the same connections, though.)

So, you might know that only signal wires b and g are turned on, but that doesn't mean segments b and g are turned on: the only digit that uses two segments is 1, so it must mean segments c and f are meant to be on. With just that information, you still can't tell which wire (b/g) goes to which segment (c/f). For that, you'll need to collect more information.

For each display, you watch the changing signals for a while, make a note of all ten unique signal patterns you see, and then write down a single four digit output value (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.

For example, here is what you might see in a single entry in your notes:

```
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf
```

(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)

Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because 7 is the only digit that uses three segments, dab in the above example means that to render a 7, signal lines d, a, and b are on. Because 4 is the only digit that uses four segments, eafb means that to render a 4, signal lines e, a, f, and b are on.

Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (cdfeb fcadb cdfeb cdbaf) use five segments and are more difficult to deduce.

For now, focus on the easy digits. Consider this larger example:

```
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
fgae cfgab fg bagce
```

Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [ ]:
def d8_entry(data):
    patterns, out = tuple(data.split(" | "))
    return [
        as_list(set, sep=" ")(patterns),
        as_list(set, sep=" ")(out),
    ]

parse = as_list(d8_entry, sep="\n")

In [ ]:
test_data_small = d8_entry("acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf")

test_data = parse("""
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
""")

data = parse(dayfile(8))

In [ ]:
test_data_small

In [ ]:
def canon(e):
    return "".join(sorted(list(e)))

def d8_solve(entry):
    patterns = entry[0]
    out = entry[1]

    patterns.sort(key=len)

    solved = {
        1: patterns[0],
        7: patterns[1],
        4: patterns[2],
        8: patterns[9],
    }

    cand_5_seg = patterns[3:6]  # 2, 3, 5
    cand_6_seg = patterns[6:9]  # 0, 6, 9

    # find 3
    for e in cand_5_seg.copy():
        if len(e.intersection(solved[1])) == 2:
            solved[3] = e
            cand_5_seg.remove(e)
            break

    for e in cand_6_seg.copy():
        if len(e.intersection(solved[1])) == 1:
            solved[6] = e
            cand_6_seg.remove(e)
        if len(e.intersection(solved[3])) == len(solved[3]):
            solved[9] = e
            cand_6_seg.remove(e)
    solved[0] = cand_6_seg[0]

    if len(cand_5_seg[0].intersection(solved[9])) == len(cand_5_seg[0]):
        solved[5] = cand_5_seg[0]
        solved[2] = cand_5_seg[1]
    else:
        solved[2] = cand_5_seg[0]
        solved[5] = cand_5_seg[1]

    mapping = {canon(v): k for k, v in solved.items()}

    return [mapping[canon(e)] for e in out]

def d8_1(data):
    counts = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for entry in data:
        for e in d8_solve(entry):
            counts[e] += 1
    return counts[1] + counts[4] + counts[7] + counts[8]

In [ ]:
d8_1(test_data)

In [ ]:
d8_1(data)

## Part 2

Through a little deduction, you should now be able to determine the remaining digits. Consider again the first example above:

acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf

After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:

 dddd
e    a
e    a
 ffff
g    b
g    b
 cccc

So, the unique signal patterns would correspond to the following digits:

- acedgfb: 8
- cdfbe: 5
- gcdfa: 2
- fbcad: 3
- dab: 7
- cefabd: 9
- cdfgeb: 6
- eafb: 4
- cagedb: 0
- ab: 1

Then, the four digits of the output value can be decoded:

- cdfeb: 5
- fcadb: 3
- cdfeb: 5
- cdbaf: 3

Therefore, the output value for this entry is 5353.

Following this same process for each entry in the second, larger example above, the output value of each entry can be determined:

- fdgacbe cefdb cefbgd gcbe: 8394
- fcgedb cgb dgebacf gc: 9781
- cg cg fdcagb cbg: 1197
- efabcd cedba gadfec cb: 9361
- gecf egdcabf bgf bfgea: 4873
- gebdcfa ecba ca fadegcb: 8418
- cefg dcbef fcge gbcadfe: 4548
- ed bcgafe cdgba cbgef: 1625
- gbdfcae bgc cg cgb: 8717
- fgae cfgab fg bagce: 4315

Adding all of the output values in this larger example produces 61229.

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [ ]:
def d8_2(data):
    total = 0
    for entry in data:
        total += int("".join(map(str, d8_solve(entry))))
    return total

In [ ]:
d8_2(test_data)

In [ ]:
d8_2(data)

# Day 9: Smoke Basin

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?



In [ ]:
parse = as_matrix(int)

In [ ]:
test_data = parse("""
2199943210
3987894921
9856789892
8767896789
9899965678
""")

data = parse(dayfile(9))

In [ ]:
def adjacency(x, y, xmax, ymax):
    candidates = [
        (x - 1, y),
        (x + 1, y),
        (x, y - 1),
        (x, y + 1),
    ]
    return [
        (i, j) for i, j in candidates
        if i >= 0 and i < xmax and j >= 0 and j < ymax
    ]

def low_points(data):
    low_points = []
    xmax = data.shape[0]
    ymax = data.shape[1]
    for i in range(xmax):
        for j in range(ymax):
            val = data[i, j]
            lowest = True
            for i_adj, j_adj in adjacency(i, j, xmax, ymax):
                if data[i_adj, j_adj] <= val:
                    lowest = False
            if lowest:
                low_points.append((i, j))
    return low_points
        

def d9_1(data):
    total_risk = 0
    for x, y in low_points(data):
        total_risk += data[x, y] + 1
                
    return total_risk

In [ ]:
d9_1(test_data)

In [ ]:
d9_1(data)

## Part 2

Next, you need to find the largest basins so you know what areas are most important to avoid.

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size 3:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The top-right basin, size 9:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The middle basin, size 14:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The bottom-right basin, size 9:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

In [ ]:
def climb_up(data, x, y):
    res = set([(x, y)])
    xmax = data.shape[0]
    ymax = data.shape[1]
    candidates = adjacency(x, y, xmax, ymax)
    val = data[x, y]
    for xe, ye in candidates:
        val_e = data[xe, ye]
        if  val_e > val and val_e < 9:
            res = res.union(climb_up(data, xe, ye))
    return res

In [ ]:
def d9_2(data):
    basins = []
    for x, y in low_points(data):
        basins.append(len(climb_up(data, x, y)))
    basins.sort()
    return basins[-3] * basins[-2] * basins[-1]

In [ ]:
d9_2(test_data)

In [ ]:
d9_2(data)

# Day 10: Syntax Scoring

You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

```
Syntax error in navigation subsystem on line: all of them
```

All of them?! The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

- If a chunk opens with (, it must close with ).
- If a chunk opens with [, it must close with ].
- If a chunk opens with {, it must close with }.
- If a chunk opens with <, it must close with >.

So, () is a legal chunk that contains no other chunks, as is []. More complex but valid chunks include ([]), {()()()}, <([{}])>, [<>({}){}[([])<>]], and even (((((((((()))))))))).

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

For example, consider the following navigation subsystem:

```
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
```

Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

- {([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
- [[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
- [{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
- [<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
- <{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.

Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

```
): 3 points.
]: 57 points.
}: 1197 points.
>: 25137 points.
```

In the above example, an illegal ) was found twice (2*3 = 6 points), an illegal ] was found once (57 points), an illegal } was found once (1197 points), and an illegal > was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?



In [ ]:
parse = as_list(str)

test_entry_good = "{}"
test_entry_corrupted = "[<(<(<(<{}))><([]([]()"

test_data = parse("""
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
""")

data = parse(dayfile(10))

In [ ]:
def d10_corrupted(entry):
    matching = {
        "(": ")",
        "[": "]",
        "{": "}",
        "<": ">",
    }
    opening = []
    for e in entry:
        if e in matching:
            opening.append(e)
        else:
            if matching[opening[-1]] != e:
                return e
            opening.pop()
    return False
    

assert not d10_corrupted(test_entry_good)
assert d10_corrupted(test_entry_corrupted) == ')'

In [ ]:
def d10_score(char):
    if char == ")":
        return 3
    if char == "]":
        return 57 
    if char == "}":
        return 1197
    if char == ">":
        return 25137
    
    return 0

def d10_1(data):
    res = 0
    for e in data:
        res += d10_score(d10_corrupted(e))
    return res

In [ ]:
d10_1(data)

## Part 2
Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In the example above, there are five incomplete lines:

 - [({(<(())[]>[[{[]{<()<>> - Complete by adding }}]])})].
 - [(()[<>])]({[<{<<[]>>( - Complete by adding )}>]}).
 - (((({<>}<{<{<>}{[]{[]{} - Complete by adding }}>}>)))).
 - {<[[]]>}<{[{[{[]{()[[[] - Complete by adding ]]}}]}]}>.
 - <{([{{}}[<[[[<>{}]]]>[]] - Complete by adding ])}>.

Did you know that autocomplete tools also have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

    ): 1 point.
    ]: 2 points.
    }: 3 points.
    >: 4 points.

So, the last completion string above - ])}> - would be scored as follows:

- Start with a total score of 0.
- Multiply the total score by 5 to get 0, then add the value of ] (2) to get a new total score of 2.
- Multiply the total score by 5 to get 10, then add the value of ) (1) to get a new total score of 11.
- Multiply the total score by 5 to get 55, then add the value of } (3) to get a new total score of 58.
- Multiply the total score by 5 to get 290, then add the value of > (4) to get a new total score of 294.

The five lines' completion strings have total scores as follows:

 - }}]])})] - 288957 total points.
 - )}>]}) - 5566 total points.
 - }}>}>)))) - 1480781 total points.
 - ]]}}]}]}> - 995444 total points.
 - ])}> - 294 total points.

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [ ]:
def d10_prefix(entry):
    matching = {
        "(": ")",
        "[": "]",
        "{": "}",
        "<": ">",
    }
    opening = []
    for e in entry:
        if e in matching:
            opening.append(e)
        elif len(opening) > 0:
            opening.pop()
        else:
            break
    return opening
        
def d10_prefix_score(opening):
    matching = {
        "(": 1,
        "[": 2,
        "{": 3,
        "<": 4,
    }
    opening.reverse()
    score = 0
    for e in opening:
        score *= 5
        score += matching[e]
    return score

def d10_2(data):
    data = filter(lambda x: not d10_corrupted(x), data)
    scores = []
    for e in data:
        scores.append(d10_prefix_score(d10_prefix(e)))
    scores = sorted(scores)
    return scores[int(len(scores) / 2)]

In [ ]:
d10_2(test_data)

In [ ]:
d10_2(data)

# Day 11: Dumbo Octopus

You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.

There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.

Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:

```
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
```

The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.

You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

- First, the energy level of each octopus increases by 1.
- Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
- Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.

Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

```
Before any steps:
11111
19991
19191
19991
11111

After step 1:
34543
40004
50005
40004
34543

After step 2:
45654
51115
61116
51115
45654
```

An octopus is highlighted when it flashed during the given step.

Here is how the larger example above progresses:

```
Before any steps:
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

After step 1:
6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637

After step 2:
8807476555
5089087054
8597889608
8485769600
8700908800
6600088989
6800005943
0000007456
9000000876
8700006848

After step 3:
0050900866
8500800575
9900000039
9700000041
9935080063
7712300000
7911250009
2211130000
0421125000
0021119000

After step 4:
2263031977
0923031697
0032221150
0041111163
0076191174
0053411122
0042361120
5532241122
1532247211
1132230211

After step 5:
4484144000
2044144000
2253333493
1152333274
1187303285
1164633233
1153472231
6643352233
2643358322
2243341322

After step 6:
5595255111
3155255222
3364444605
2263444496
2298414396
2275744344
2264583342
7754463344
3754469433
3354452433

After step 7:
6707366222
4377366333
4475555827
3496655709
3500625609
3509955566
3486694453
8865585555
4865580644
4465574644

After step 8:
7818477333
5488477444
5697666949
4608766830
4734946730
4740097688
6900007564
0000009666
8000004755
6800007755

After step 9:
9060000644
7800000976
6900000080
5840000082
5858000093
6962400000
8021250009
2221130009
9111128097
7911119976

After step 10:
0481112976
0031112009
0041112504
0081111406
0099111306
0093511233
0442361130
5532252350
0532250600
0032240000
```

After step 10, there have been a total of 204 flashes. Fast forwarding, here is the same configuration every 10 steps:

```
After step 20:
3936556452
5686556806
4496555690
4448655580
4456865570
5680086577
7000009896
0000000344
6000000364
4600009543

After step 30:
0643334118
4253334611
3374333458
2225333337
2229333338
2276733333
2754574565
5544458511
9444447111
7944446119

After step 40:
6211111981
0421111119
0042111115
0003111115
0003111116
0065611111
0532351111
3322234597
2222222976
2222222762

After step 50:
9655556447
4865556805
4486555690
4458655580
4574865570
5700086566
6000009887
8000000533
6800000633
5680000538

After step 60:
2533334200
2743334640
2264333458
2225333337
2225333338
2287833333
3854573455
1854458611
1175447111
1115446111

After step 70:
8211111164
0421111166
0042111114
0004211115
0000211116
0065611111
0532351111
7322235117
5722223475
4572222754

After step 80:
1755555697
5965555609
4486555680
4458655580
4570865570
5700086566
7000008666
0000000990
0000000800
0000000000

After step 90:
7433333522
2643333522
2264333458
2226433337
2222433338
2287833333
2854573333
4854458333
3387779333
3333333333

After step 100:
0397666866
0749766918
0053976933
0004297822
0004229892
0053222877
0532222966
9322228966
7922286866
6789998766
```

After 100 steps, there have been a total of 1656 flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

In [ ]:
parse = as_matrix(int)

test_data = parse("""
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
""")

data = parse(dayfile(11))

In [ ]:
def d11_update(data, x, y):
    if data[x, y] < 0: # flashed this turn
        return
    data[x, y] += 1
    if data[x, y] > 9: # flash!
        data[x, y] = -1
        for i, j in [
            [-1, -1], [-1, 0], [-1, 1],
            [ 0, -1],          [ 0, 1],
            [ 1, -1], [ 1, 0], [ 1, 1],
        ]:
            if (    x + i >= 0 and x + i < data.shape[0]
                and y + j >= 0 and y + j < data.shape[0]):
                d11_update(data, x + i, y + j)

def d11_step(data):
    data = data.copy()
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            d11_update(data, x, y)
            
    flashes = 0
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            if data[x, y] == -1:
                flashes += 1
                data[x, y] = 0
                
    return data, flashes

def d11_1(data, nb_steps = 100, debug = False):
    nb_flashes = 0
    for i in range(nb_steps):
        data, flashes_i = d11_step(data)
        nb_flashes += flashes_i
        if debug:
            print(f"{i=} {flashes_i}")
    return nb_flashes

In [ ]:
d11_1(test_data)

In [ ]:
d11_1(data)

## Part Two

It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be synchronizing!

In the example above, the first time all octopuses flash simultaneously is step 195:

```
After step 193:
5877777777
8877777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777

After step 194:
6988888888
9988888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888

After step 195:
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
```

If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. What is the first step during which all octopuses flash?



In [ ]:
def d11_2(data):
    i = 0
    while True:
        i += 1
        data, flashes_i = d11_step(data)
        if flashes_i == data.size:
            return i

In [ ]:
d11_2(test_data)

In [ ]:
d11_2(data)

# Day 12: Passage Pathing

With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the best path is to find all of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

```
start-A
start-b
A-c
A-b
b-d
A-end
b-end
```

This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

So, the above cave system looks roughly like this:

```
    start
    /   \
c--A-----b--d
    \   /
     end
```

Your goal is to find the number of distinct paths that start at start, end at end, and don't visit small caves more than once. There are two types of caves: big caves (written in uppercase, like A) and small caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should visit small caves at most once, and can visit big caves any number of times.

Given these rules, there are 10 paths through this example cave system:

```
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
```

(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

Here is a slightly larger example:

```
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
```

The 19 paths through it are as follows:

```
start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
```

Finally, this even larger example has 226 paths through it:

```
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
```

How many paths through this cave system are there that visit small caves at most once?

In [ ]:
def as_link(entry):
    a, b = entry.split("-")
    return (a, b)

def parse(data):
    links = as_list(as_link)(data)
    res = {}
    for a, b in links:
        if a not in res:
            res[a] = set()
        if b not in res:
            res[b] = set()
        res[a].add(b)
        res[b].add(a)
    return res
    return


In [ ]:
test_data = parse("""
start-A
start-b
A-c
A-b
b-d
A-end
b-end
""")


test2_data = parse("""
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
""")

data = parse(dayfile(12))

In [ ]:
def is_small(node):
    l = node[0]
    return l == l.lower()
    
def possible_paths(links, node, small_explored):
    if node == "end":
        return [[node]]
    
    if is_small(node):
        small_explored.add(node)

    res = []
    for candidate in links[node].difference(small_explored):
        res = res + [[node] + e for e in possible_paths(links, candidate, small_explored.copy())]

    return res
    
def d12_1(data):
    return len(possible_paths(data, "start", set()))

In [ ]:
d12_1(test_data)

In [ ]:
d12_1(test2_data)

In [ ]:
d12_1(data)

## Part 2

After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.

Now, the 36 possible paths through the first example above are:

```
start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
```

The slightly larger example above now has 103 paths through it, and the even larger example now has 3509 paths through it.

Given these new rules, how many paths through this cave system are there?

In [ ]:
def possible_paths2(links, node, small_explored, already_taken_twice):
    if node == "end":
        return [[node]]
    
    if is_small(node):
        small_explored.add(node)

    res = []
    for candidate in links[node]:
        if candidate == "start":
            continue
        if candidate in small_explored and already_taken_twice:
            continue
        res = res + [[node] + e for e in possible_paths2(links, candidate, small_explored.copy(), (candidate in small_explored or already_taken_twice))]

    return res

def d12_2(data):
    return len(possible_paths2(data, "start", set(), False))

In [ ]:
d12_2(test_data)

In [ ]:
d12_2(test2_data)

In [ ]:
d12_2(data)

# Day 13: Transparent Origami

You reach another volcanically active part of the cave. It would be nice if you could do some kind of thermal imaging so you could tell ahead of time which caves are too hot to safely enter.

Fortunately, the submarine seems to be equipped with a thermal camera! When you activate it, you are greeted with:

Congratulations on your purchase! To activate this infrared thermal imaging
camera system, please enter the code found on page 1 of the manual.

Apparently, the Elves have never used this feature. To your surprise, you manage to find the manual; as you go to open it, page 1 falls out. It's a large sheet of transparent paper! The transparent paper is marked with random dots and includes instructions on how to fold it up (your puzzle input). For example:

```
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
```

The first section is a list of dots on the transparent paper. 0,0 represents the top-left coordinate. The first value, x, increases to the right. The second value, y, increases downward. So, the coordinate 3,0 is to the right of 0,0, and the coordinate 0,7 is below 0,0. The coordinates in this example form the following pattern, where # is a dot on the paper and . is an empty, unmarked position:

```
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
```

Then, there is a list of fold instructions. Each instruction indicates a line on the transparent paper and wants you to fold the paper up (for horizontal y=... lines) or left (for vertical x=... lines). In this example, the first fold instruction is fold along y=7, which designates the line formed by all of the positions where y is 7 (marked here with -):

```
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
-----------
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
```

Because this is a horizontal line, fold the bottom half up. Some of the dots might end up overlapping after the fold is complete, but dots will never appear exactly on a fold line. The result of doing this fold looks like this:

```
#.##..#..#.
#...#......
......#...#
#...#......
.#.#..#.###
...........
...........
```

Now, only 17 dots are visible.

Notice, for example, the two dots in the bottom left corner before the transparent paper is folded; after the fold is complete, those dots appear in the top left corner (at 0,0 and 0,1). Because the paper is transparent, the dot just below them in the result (at 0,3) remains visible, as it can be seen through the transparent paper.

Also notice that some dots can end up overlapping; in this case, the dots merge together and become a single dot.

The second fold instruction is fold along x=5, which indicates this line:

```
#.##.|#..#.
#...#|.....
.....|#...#
#...#|.....
.#.#.|#.###
.....|.....
.....|.....
```

Because this is a vertical line, fold left:

```
#####
#...#
#...#
#...#
#####
.....
.....
```

The instructions made a square!

The transparent paper is pretty big, so for now, focus on just completing the first fold. After the first fold in the example above, 17 dots are visible - dots that end up overlapping after the fold is completed count as a single dot.

How many dots are visible after completing just the first fold instruction on your transparent paper?

In [ ]:
def d13_fold_instruction(entry):
    axis, value = entry.replace("fold along ", "").split("=")
    axis = 0 if axis == "x" else 1
    return (axis, int(value))

def parse(data):
    raw_coords, raw_instrs = data.split("\n\n")
    coords = set(as_list(as_pair(int), sep="\n")(raw_coords))
    instrs = as_list(d13_fold_instruction, sep="\n")(raw_instrs)
    return {
        "coords": coords,
        "instrs": instrs,
    }

In [ ]:
test_data = parse("""
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
""")

data = parse(dayfile(13))

In [ ]:
test_data

In [ ]:
def fold(coords, axis, value):
    res = set()
    for entry in coords:
        if entry[axis] <= value:
            res.add(entry)
        else:
            entry_new = list(entry)
            entry_new[axis] = 2 * value - entry_new[axis]
            res.add(tuple(entry_new))
    return res
    
def d13_1(data):
    return len(fold(data["coords"], *data["instrs"][0]))

In [ ]:
d13_1(test_data)

In [ ]:
d13_1(data)

## Part 2

Finish folding the transparent paper according to the instructions. The manual says the code is always eight capital letters.

What code do you use to activate the infrared thermal imaging camera system?

In [ ]:
def d13_display(coords):
    x_max = 0
    y_max = 0
    for x, y in coords:
        if x > x_max:
            x_max = x
        if y > y_max:
            y_max = y
    for y in range(y_max + 1):
        for x in range(x_max + 1):
            if (x, y) in coords:
                print("#", end="")
            else:
                print(".", end="")
        print()

def d13_2(data):
    coords = data["coords"]
    for instr in data["instrs"]:
        coords = fold(coords, *instr)
    d13_display(coords)

In [ ]:
d13_2(test_data)

In [ ]:
d13_2(data)

---